In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import pickle
import numpy as np

In [28]:
data_path = "../data"
ratings = pd.read_csv(data_path+'/ratings_small.csv')
movies_metadata = pd. read_csv(data_path + '/movies_metadata.csv')
links_small = pd.read_csv(data_path+'/links_small.csv')

C:\Users\TGSUser172\AppData\Local\Temp\ipykernel_4316\469106208.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd. read_csv(data_path + '/movies_metadata.csv')


In [29]:
movies_metadata = movies_metadata.drop([19730, 29503, 35587])

In [30]:
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [31]:
links_small.sample()

,movieId,imdbId,tmdbId
7045,62049,48918,1984.0


In [32]:
links_small_ids = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
movies_metadata['id'] = movies_metadata['id'].astype('int')
s_movies_metadata = movies_metadata[movies_metadata['id'].isin(links_small_ids)]

In [33]:
s_movies_metadata.shape

(9099, 24)

In [34]:
s_movies_metadata.sample()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
2924,False,"{'id': 48037, 'name': 'Meatballs Collection', ...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,40771,tt0087700,en,Meatballs Part II,The second in-name-only sequel to the first Me...,...,1984-07-27,0.0,86.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The insanity continues...,Meatballs Part II,False,3.5,13.0


In [35]:
ratings.sample()

,userId,movieId,rating,timestamp
63280,457,2997,4.0,1471383437


In [36]:
ratings['movieId'] = ratings['movieId'].astype('int')
s_ratings = ratings[ratings['movieId'].isin(links_small_ids)]

In [37]:
s_ratings.shape

(32146, 4)

In [38]:
s_movies_metadata['tagline'] = s_movies_metadata['tagline'].fillna('')
s_movies_metadata['description'] = s_movies_metadata['overview'] + s_movies_metadata['tagline']
s_movies_metadata['description'] = s_movies_metadata['description'].fillna('')

C:\Users\TGSUser172\AppData\Local\Temp\ipykernel_4316\2174596264.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_movies_metadata['tagline'] = s_movies_metadata['tagline'].fillna('')
C:\Users\TGSUser172\AppData\Local\Temp\ipykernel_4316\2174596264.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_movies_metadata['description'] = s_movies_metadata['overview'] + s_movies_metadata['tagline']
C:\Users\TGSUser172\AppData\Local\Temp\ipykernel_4316\2174596264.py:3: SettingWithCopyWarning: 
A value is tryi

In [42]:
# s_movies_metadata.sample()
# s_movies_metadata[s_movies_metadata.duplicated()]
s_movies_metadata.drop_duplicates(subset=["description"], inplace=True)

C:\Users\TGSUser172\AppData\Local\Temp\ipykernel_4316\3576824576.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s_movies_metadata.drop_duplicates(subset=["description"], inplace=True)


In [51]:
s_movies_metadata.reset_index(drop=True, inplace=True)

In [52]:
s_movies_metadata.sample()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description
5393,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",NaN,59427,tt0064893,en,Ring of Bright Water,Whoever said dog is man's best friend never me...,...,0.0,107.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,"The stars, the warmth, the pleasure of Born Free.",Ring of Bright Water,False,3.2,6.0,Whoever said dog is man's best friend never me...


In [53]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(s_movies_metadata['description'])

In [54]:
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [57]:
s_movies_metadata.index[s_movies_metadata.title == 'Grumpier Old Men'].tolist()

[2]

In [58]:
def get_recommendations(df, title):
    idx =  df.index[df.title == title][0]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return df["title"].iloc[movie_indices]

In [59]:
get_recommendations(s_movies_metadata, 'Grumpier Old Men')

5876         An Extremely Goofy Movie
2745                   Grumpy Old Men
8603         The Hundred-Foot Journey
697                      Gone Fishin'
397                          Fearless
410                           Go Fish
211                     A Goofy Movie
1340                             Bent
1897                         Rushmore
3357                    Heartbreakers
1426                   Kissing a Fool
8872          The Secret Life of Pets
3653                              Tap
8800                      The Cobbler
2801              The End of Violence
5529                       Collateral
5739             Yours, Mine and Ours
2753    Hillbillys in a Haunted House
7756                             Thor
5284                         The Trip
Name: title, dtype: object

In [62]:
ratings.sample()

,userId,movieId,viewed,timestamp
15723,102,2085,4.0,956599039


In [61]:
ratings.rename(columns = {'rating':'viewed'}, inplace = True)

In [63]:
ratings['viewed'] = np.where(ratings['viewed'] < 2.5, 0,1)
ratings["viewed"] = ratings["viewed"].astype(int)
ratings.sample(5)

,userId,movieId,viewed,timestamp
72855,509,2739,1,940015074
27074,199,1676,1,1214914656
47157,346,1200,1,1044650752
86455,578,6870,1,1418984676
79527,547,4679,1,1022678444


In [64]:
ratings[ratings.duplicated()]

,userId,movieId,viewed,timestamp


In [65]:
ratings.reset_index(inplace=True, drop=True)

In [66]:
ratingsmatrix = ratings.pivot(index="userId", columns="movieId", values="viewed").fillna(0)
matrix = ratingsmatrix.values

In [67]:
u, s, vh = np.linalg.svd(matrix, full_matrices=False)

In [68]:
def cosine_similarity(v,u):
    return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))

def custom_cosine(vh, movieIdx, top_n):
    res = []
    a = vh[:, movieIdx]
    for i in range(1, vh.shape[1]):
        if i == movieIdx:
            continue
        else:
            b = vh[:, i]
            similarity = cosine_similarity(a, b)
            if abs(similarity) > 0.5:
                res.append(i)
                if len(res) == top_n:
                    break
    return res

def get_movies(moviesIds, idxs, df):
    moviesId = [moviesIds[x] for x in idxs]
    data = df[df.id.isin(moviesId)].title.tolist()
    return data

In [78]:
k = 50
moviesId = list(set(ratings['movieId'].tolist()))
movie_id = -1
while movie_id == -1:
    sample = s_movies_metadata.sample() 
    tempId = sample.id.item()
    if tempId in ratings["movieId"].tolist():           
        idx = tempId
        movie_id = moviesId.index(idx)

print(f"Selected movie {sample.title.item()}")
top_n = 50

temp = custom_cosine(vh, movie_id, top_n)
get_movies(moviesId, temp, s_movies_metadata)

Selected movie Heavenly Creatures


C:\Users\TGSUser172\AppData\Local\Temp\ipykernel_4316\298090851.py:2: RuntimeWarning: invalid value encountered in double_scalars
  return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))


["What's Eating Gilbert Grape",
 'The Shining',
 'Benny & Joon',
 'Ghost Dog: The Way of the Samurai',
 'Shower',
 'Vivre Sa Vie',
 'Lammbock',
 'Lassie Come Home']

In [80]:
s_movies_metadata = s_movies_metadata[["id", "title"]]

In [81]:
s_movies_metadata.to_csv("../artifacts/movie_idx.csv")

In [82]:
pickle.dump(cosine_sim_matrix, open('../artifacts/cosine_matrix.pkl', 'wb'))

In [83]:
pickle.dump(vh, open('../artifacts/svd_matrix.pkl', 'wb'))

In [84]:
ratings.to_csv("../artifacts/user_data.csv", index=False)